Hospital Readmissions

In [1]:
import re
import math
import sys
import numpy as np
import pandas as pd
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from pprint import pprint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score, mean_absolute_error
from scipy import stats
from sklearn.model_selection import RepeatedKFold
import scipy
import sklearn

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 1000000)
pd.set_option('display.width', 1000000)
pd.set_option('display.max_colwidth', 1000000)
pd.set_option('display.max_info_columns', 1000)
np.set_printoptions(threshold=sys.maxsize)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import six

In [2]:
df = pd.read_csv('diabetic_data.csv', header=0, index_col=0)

In [3]:
import qgrid
qgrid.nbinstall(overwrite=True)

AttributeError: module 'qgrid' has no attribute 'nbinstall'

In [ ]:
df.head()
#df.info()
#df.nunique()

import ipysheet

from ipysheet import from_dataframe
sheet = from_dataframe(df.iloc[0:10, 0:3])
display(sheet)

In [ ]:
qw = qgrid.show_grid(df.iloc[:10, :4], show_toolbar=False, grid_options={'forceFitColumns': False, 'fullWidthRows': True})
qw